<a href="https://colab.research.google.com/github/masoudsoroush/DATA-690-WANG/blob/master/Assignments/Assignment-06/Homework-06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 06 - Using Pandas
## Instructions
- This assignment is a re-do of assignment 04. We use Pandas and functions instead. It is due Wednesday, Sept 30th.
- WYou get to use the simple and convenient read_html() from Pandas to extract **HTML tables** from web pages. 
- We had walked through the process during last sesssion. The sample solution is here:
- https://github.com/wcj365/python-stats-dataviz/blob/master/pandas_apply.ipynb
- You will try to do it on your own and use the sample solution when you get stuck. 
- Make sure you user **Markdown cells** to document the process and its steps.
## Requirements
1. Use Pandas read_html() function to retrive the HTML tables as dataframes from the web page: https://www.genealogybranches.com/censuscosts.html
2. Find out how many HTML tables Pandas retrieves from the web page. Find out which dataframe contain the data and use it for further processing.
3. Display and explore the data (rows, columns, etc.) and determine the data quality (bad rows, bad columns, bad elements, null values, etc.)
4. Document the data quality issues using a Markdown cell so the reader understands what the problems are.
5. Write a function named cleanse_year() that takes a string as input and removes any asterisks in the string and return the cleansed string. Test the function by using test strings (for example, "1989*", "20*10")
6. Cleanse the column "Census Year" using the function cleanse_year() and Pandas's apply() function.
7. Repeat this process for column "Total Population", 	"Census Cost", 	"Average Cost Per Person". (define, test, and apply the function)
8. After all columns are cleansed, save the clenased dataframe to a file named "census_cost_cleansed.csv" using CSV format.
9. Use Pandas to read the saved cleansed file and explore to make sure it is clean. 
10. Updload both the Jupyter notebook and the cleanse data file to GitHub and submit the GitHub link in BlackBoard.

> Give yourself a pad on the back. Now you get a feel of how wonderful Pandas is and 
> how powerful the combination of apply() and user-defined functions are.


In [1]:
import pandas as pd

Reading the tables directly from the given website 

In [2]:
list_census=pd.read_html('https://www.genealogybranches.com/censuscosts.html')

Figuring out how many tables are read from the website

In [3]:
len(list_census)

2

It is realized that the second table is not relevant for our analysis

In [4]:
list_census[1]

,0
0,"The U.S. population reached 300,000,000 on 17 ..."


We focus on the first table to analyze the data.

In [5]:
df=list_census[0]
df.head()

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


In [6]:
df.iloc[0]

0                Census Year
1           Total Population
2                Census Cost
3    Average Cost Per Person
Name: 0, dtype: object

Replacing the columns by the first row, and getting rid of the first row.

In [7]:
df.columns=df.iloc[0]
df.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


In [8]:
df.drop([0],inplace=True)
df.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents


Write a function named cleanse_year() that takes a string as input and removes any asterisks in the string and return the cleansed string. Test the function by using test strings (for example, "1989*", "20*10")

In [9]:
# The 'Total Population' column is already clean. We only convert the values to integers.

df['Total Population']=df['Total Population'].apply(int)

In [10]:
# The following function gets rid of any asterisks, and converts the value to an integer.

def cleanse_year(x):
  y=x
  for k in x:
    if k=='*':
      y=int(x.replace('*',''))
  return y

In [11]:
# Testing the 'cleanse_year' function.

print(cleanse_year('1989*'))
print(cleanse_year('21*10'))

1989
2110


Cleanse the column "Census Year" using the function cleanse_year() and Pandas's apply() function.

In [12]:
# Cleansing the 'Census Year' column of the dataframe.

df['Census Year']=df['Census Year'].apply(cleanse_year)

Repeat this process for column "Total Population", 	"Census Cost", 	"Average Cost Per Person". (define, test, and apply the function)

In [13]:
# This function automatically removes the '.' uniformly (I used the same function I had defined in HW-04).

def RemoveDot (string):
    a=0; x=string;
    b=x.find('.')
    if b !=-1: 
        for k in range(len(x)-b):
            if x[len(x)-k-1]=='0':
                a += 1
            else:
                break
        y=x.replace('.','')[:a+b] + '.' + x.replace('.','')[a+b:]
    else:
        y=x
    return y

In [14]:
# The following function cleanse the cost.

def cleanse_cost(x):
  y=x.replace('$','').replace(',','').replace(' Billion','000000000')
  y=int(float(RemoveDot(y))) 
  return y

In [15]:
# Testing the above function.

print(cleanse_cost('$3,421,198'))
print(cleanse_cost('$3.345237 Billion'))
print(cleanse_cost('$13 Billion'))

3421198
3345237000
13000000000


In [16]:
# Cleansing the 'Census Cost' column of the dataframe.

df['Census Cost']=df['Census Cost'].apply(cleanse_cost)

In [17]:
# The following function removes 'cents' and divides by 100.

def RemoveCents (string):
    x=string.replace('$','')
    b=x.find(' cents')
    if b !=-1: 
        y=float(x.replace(' cents','')) * 0.01
    else:
        y=float(x)
    return y

In [18]:
# Testing the above function.

print(RemoveCents('2.16 cents'))
print(RemoveCents('$4.76'))

0.0216
4.76


In [19]:
# Cleansing the 'Average Cost Per Person' column of the dataframe.

df['Average Cost Per Person']=df['Average Cost Per Person'].apply(RemoveCents)

After all columns are cleansed, save the clenased dataframe to a file named "census_cost_cleansed.csv" using CSV format.

In [20]:
# Saving the cleansed dataframe as a csv file.

df.to_csv('census_cost_cleansed.csv')

Use Pandas to read the saved cleansed file and explore to make sure it is clean.

In [21]:
# Reading the saved csv file.

New_df=pd.read_csv('census_cost_cleansed.csv')

In [22]:
# I am not sure why a new column is added to the new dataframe when it is read from the csv file! I just get rid 
# of that unwanted column. Dr. Wang, any idea why this happens?

New_df.drop(columns=['Unnamed: 0'],inplace=True)

In [23]:
# Exploring for any anomalies. 

New_df

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,1790,3929214,44377,0.0113
1,1800,5308483,66109,0.0124
2,1810,7239881,178445,0.0246
3,1820,9633822,208526,0.0216
4,1830,12866020,378545,0.0294
5,1840,17069458,833371,0.0488
6,1850,23191876,1423351,0.0614
7,1860,31443321,1969377,0.0626
8,1870,38558371,3421198,0.0887
9,1880,50155783,5790678,0.1154


In [24]:
# Statistical indicators of all columns are displayed!

New_df.describe()

,Census Year,Total Population,Census Cost,Average Cost Per Person
count,23.0000,2.300000e+01,2.300000e+01,23.000000
mean,1900.0000,1.039211e+08,9.445587e+08,3.368739
std,67.8233,9.622629e+07,2.830037e+09,9.278241
min,1790.0000,3.929214e+06,4.437700e+04,0.011300
25%,1845.0000,2.013067e+07,1.128361e+06,0.055100
50%,1900.0000,7.630339e+07,1.185400e+07,0.170700
75%,1955.0000,1.653245e+08,1.096980e+08,0.658900
max,2010.0000,3.087455e+08,1.300000e+10,42.110000
